In [1]:
import torch
import time
from torch import nn

# === импорт твоего слоя INT8 ===
from conv_gemm.layers.triton_conv2d_int8 import TritonConv2dINT8

In [2]:
torch.manual_seed(0)
device = "cuda"

In [11]:
N = 1
Cin = 3
Cout = 8
H = W = 32
K = 3

x = torch.randn(N, Cin, H, W, device=device, dtype=torch.float32)
conv_ref = nn.Conv2d(Cin, Cout, K, padding=K//2, bias=True).to(device)

In [12]:
conv_int8 = TritonConv2dINT8(
    Cin, Cout, K,
    padding=K//2,
    precision_mode="int8_infer"   # чистый INT8 инференс
).to(device)

# base forvard

In [13]:
with torch.no_grad():
    conv_int8.weight.copy_(conv_ref.weight)
    if conv_ref.bias is not None:
        conv_int8.bias.copy_(conv_ref.bias)

In [14]:
# FP32 reference
y_ref = conv_ref(x)

# INT8 Triton
y_int8 = conv_int8(x)

print("y_ref.shape:", y_ref.shape)
print("y_int8.shape:", y_int8.shape)

y_ref.shape: torch.Size([1, 8, 32, 32])
y_int8.shape: torch.Size([1, 8, 32, 32])


In [7]:
# ============================================================
#                   ОЦЕНКА ТОЧНОСТИ
# ============================================================

err = (y_ref - y_int8).abs()
print("\n=== ACCURACY CHECK ===")
print("max error:", err.max().item())
print("mean error:", err.mean().item())


=== ACCURACY CHECK ===
max error: 0.01782292127609253
mean error: 0.0037985797971487045


In [8]:
def bench(fn, iters=50):
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(iters):
        fn()
    torch.cuda.synchronize()
    return (time.time() - start) * 1000 / iters  # ms

t_ref = bench(lambda: conv_ref(x))
t_int8 = bench(lambda: conv_int8(x))

print("\n=== SPEED (ms) ===")
print(f"PyTorch FP32 Conv2D:   {t_ref:.3f} ms")
print(f"Triton INT8 Conv2D:    {t_int8:.3f} ms")
print(f"Speedup: {t_ref / t_int8:.3f}x")


=== SPEED (ms) ===
PyTorch FP32 Conv2D:   0.049 ms
Triton INT8 Conv2D:    0.669 ms
Speedup: 0.073x


# BENCH FORVARD

In [1]:
import torch
import time
import pandas as pd
from torch import nn

from conv_gemm.layers.triton_conv2d_int8 import TritonConv2dINT8  # путь подгони под себя

device = "cuda"
torch.manual_seed(0)


def bench_ms(fn, iters=50):
    torch.cuda.synchronize()
    t0 = time.time()
    for _ in range(iters):
        fn()
    torch.cuda.synchronize()
    return (time.time() - t0) * 1000.0 / iters


def run_int8_conv_bench(
    image_sizes=(32, 64, 112, 224, 512),
    batch_sizes=(1, 2, 3, 4),
    channels=((1, 1), (1, 3), (3, 8), (8, 16), (16, 32)),
    kernels=(1, 3, 5, 7, 9, 11),
    iters=50,
):
    """
    channels: кортеж пар (Cin, Cout),
      например: ((1,1), (1,3), (3,8), (8,16), (16,32))
    """
    rows = []

    for H in image_sizes:
        W = H
        for N in batch_sizes:
            for (Cin, Cout) in channels:
                for K in kernels:
                    # Нормальная свёртка не умеет kernel > spatial
                    if K > H or K > W:
                        continue

                    print(f"[bench] img={H} N={N} Cin={Cin} Cout={Cout} K={K}")

                    # Создаём вход и две свёртки
                    x = torch.randn(N, Cin, H, W, device=device, dtype=torch.float32)

                    conv_ref = nn.Conv2d(
                        Cin, Cout, kernel_size=K,
                        stride=1,
                        padding=K // 2,
                        bias=True,
                    ).to(device)

                    conv_int8 = TritonConv2dINT8(
                        in_channels=Cin,
                        out_channels=Cout,
                        kernel_size=K,
                        stride=1,
                        padding=K // 2,
                        dilation=1,
                        bias=True,
                        precision_mode="int8_infer",   # чистый инференс
                        use_weight_shadow=False,
                    ).to(device)

                    # Копируем веса/биасы, чтобы честно сравнивать
                    with torch.no_grad():
                        conv_int8.weight.copy_(conv_ref.weight)
                        if conv_ref.bias is not None:
                            conv_int8.bias.copy_(conv_ref.bias)

                    # FP32 reference
                    try:
                        with torch.no_grad():
                            y_ref = conv_ref(x)
                    except Exception as e:
                        print("  [SKIP] FP32 conv failed:", e)
                        rows.append([H, N, Cin, Cout, K,
                                     None, None, None, None, None, str(e)])
                        continue

                    # INT8 forward (один прогон для ошибки)
                    try:
                        with torch.no_grad():
                            y_int8 = conv_int8(x)
                    except Exception as e:
                        print("  [SKIP] INT8 conv failed:", e)
                        rows.append([H, N, Cin, Cout, K,
                                     None, None, None, None, None, f"int8_fail: {e}"])
                        continue

                    # Ошибки
                    err = (y_ref - y_int8).abs()
                    err_max = err.max().item()
                    err_mean = err.mean().item()

                    # Бенч
                    try:
                        t_ref = bench_ms(lambda: conv_ref(x), iters=iters)
                        t_int8 = bench_ms(lambda: conv_int8(x), iters=iters)
                        speedup = t_ref / t_int8 if t_int8 > 0 else None
                    except Exception as e:
                        print("  [WARN] bench failed:", e)
                        t_ref = t_int8 = speedup = None

                    rows.append([
                        H, N, Cin, Cout, K,
                        t_ref, t_int8, speedup,
                        err_max, err_mean,
                        None,
                    ])

    df = pd.DataFrame(
        rows,
        columns=[
            "img", "N", "Cin", "Cout", "K",
            "t_ref_ms", "t_int8_ms", "speedup",
            "err_max", "err_mean", "note",
        ],
    )
    return df




In [2]:
channels_cfg = (
    (1, 1),
    (1, 3),
    (3, 8),
    (8, 16),
    (16, 32),
    (32, 64),
)

df = run_int8_conv_bench(
    image_sizes=(32, 64, 112, 224),
    batch_sizes=(1, 2, 4, 8),
    channels=channels_cfg,
    kernels=(1, 3, 5, 7, 9, 11),
    iters=30,
)

[bench] img=32 N=1 Cin=1 Cout=1 K=1
[bench] img=32 N=1 Cin=1 Cout=1 K=3
[bench] img=32 N=1 Cin=1 Cout=1 K=5
[bench] img=32 N=1 Cin=1 Cout=1 K=7
[bench] img=32 N=1 Cin=1 Cout=1 K=9
[bench] img=32 N=1 Cin=1 Cout=1 K=11
[bench] img=32 N=1 Cin=1 Cout=3 K=1
[bench] img=32 N=1 Cin=1 Cout=3 K=3
[bench] img=32 N=1 Cin=1 Cout=3 K=5
[bench] img=32 N=1 Cin=1 Cout=3 K=7
[bench] img=32 N=1 Cin=1 Cout=3 K=9
[bench] img=32 N=1 Cin=1 Cout=3 K=11
[bench] img=32 N=1 Cin=3 Cout=8 K=1
[bench] img=32 N=1 Cin=3 Cout=8 K=3
[bench] img=32 N=1 Cin=3 Cout=8 K=5
[bench] img=32 N=1 Cin=3 Cout=8 K=7
[bench] img=32 N=1 Cin=3 Cout=8 K=9
[bench] img=32 N=1 Cin=3 Cout=8 K=11
[bench] img=32 N=1 Cin=8 Cout=16 K=1
[bench] img=32 N=1 Cin=8 Cout=16 K=3
[bench] img=32 N=1 Cin=8 Cout=16 K=5
[bench] img=32 N=1 Cin=8 Cout=16 K=7
[bench] img=32 N=1 Cin=8 Cout=16 K=9
[bench] img=32 N=1 Cin=8 Cout=16 K=11
[bench] img=32 N=1 Cin=16 Cout=32 K=1
[bench] img=32 N=1 Cin=16 Cout=32 K=3
[bench] img=32 N=1 Cin=16 Cout=32 K=5
[bench] img=

In [3]:
# Топ по ускорению среди конфигов, где всё отработало
df_valid = df.dropna(subset=["t_ref_ms", "t_int8_ms", "speedup"])
df_top = df_valid.sort_values("speedup", ascending=False).head(30)
df_top

,img,N,Cin,Cout,K,t_ref_ms,t_int8_ms,speedup,err_max,err_mean,note
242,64,4,16,32,5,0.718951,0.610677,1.177301,0.032164,0.005532,None
278,64,8,16,32,5,0.562366,1.079345,0.521025,0.033245,0.005173,None
557,224,8,3,8,11,2.267710,8.513379,0.266370,0.034025,0.005471,None
485,224,2,3,8,11,0.634424,2.461735,0.257714,0.031714,0.005409,None
70,32,2,32,64,9,0.140158,0.617409,0.227011,0.026461,0.004531,None
177,64,1,32,64,7,0.138942,0.613379,0.226519,0.027582,0.004607,None
521,224,4,3,8,11,0.971309,4.297113,0.226038,0.034512,0.005599,None
71,32,2,32,64,11,0.173664,0.794903,0.218472,0.028644,0.004764,None
413,112,8,3,8,11,0.512234,2.350140,0.217959,0.029564,0.004864,None
556,224,8,3,8,9,1.335343,6.218743,0.214729,0.032571,0.005144,None


# base backvard

In [5]:
def bench_backward_ms(module, x, iters=50):
    """
    Меряем время полного прохода: forward + backward по loss = y.sum().
    Градиенты по весам и входу считаются, но не используются.
    """
    torch.cuda.synchronize()
    t0 = time.time()
    for _ in range(iters):
        # новый вход с requires_grad, чтобы каждый раз строился граф
        x_in = x.detach().clone().requires_grad_(True)
        module.zero_grad(set_to_none=True)

        y = module(x_in)
        loss = y.sum()
        loss.backward()
    torch.cuda.synchronize()
    return (time.time() - t0) * 1000.0 / iters  # ms

In [6]:
# ==== CHECK GRADS (один прогон) ====
x_ref = x.detach().clone().requires_grad_(True)
x_int = x.detach().clone().requires_grad_(True)

conv_ref.zero_grad(set_to_none=True)
conv_int8.zero_grad(set_to_none=True)

y_ref = conv_ref(x_ref)
y_int = conv_int8(x_int)

loss_ref = y_ref.sum()
loss_int = y_int.sum()

loss_ref.backward()
loss_int.backward()

# градиенты по входу
dx_err = (x_ref.grad - x_int.grad).abs()
dx_err_max = dx_err.max().item()
dx_err_mean = dx_err.mean().item()

# градиенты по весам
dw_err = (conv_ref.weight.grad - conv_int8.weight.grad).abs()
dw_err_max = dw_err.max().item()
dw_err_mean = dw_err.mean().item()

print("\n=== GRAD CHECK ===")
print(f"dX  max err: {dx_err_max:.6e}, mean err: {dx_err_mean:.6e}")
print(f"dW  max err: {dw_err_max:.6e}, mean err: {dw_err_mean:.6e}")


CompilationError: at 38:15:
            mask=mask_a, other=0
        )
        b = tl.load(
            B_ptr + offs_k[:, None] * stride_bk + offs_n[None, :] * stride_bn,
            mask=mask_b, other=0
        )

        if USE_FP16:
            a = a.to(tl.float16)
            b = b.to(tl.float16)
        # acc остаётся fp32
        acc += tl.dot(a, b, allow_tf32=False)
               ^